In [ ]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

COMMIT_HASH = "087df4f"

In [ ]:
df = pd.read_csv("../test/test_docking.csv")
df.scores = df.scores.apply(lambda x: min(x,0))
df.commit_hash [(df.method == "pyscreener")]= COMMIT_HASH
df=df[df.commit_hash == COMMIT_HASH]

df["time_per_molecule_s"] = df.time_per_molecule_s / df.exhaustiveness

df

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(7,4))

# ax in log scale
ax = axes[0]
sns.boxplot(df.sort_values("time_docking_s", ascending=False), x="exhaustiveness", y="time_per_molecule_s", hue="method", dodge=False, ax=ax)
ax.set_yscale("log")
# More yticks
yticks = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,2,4,5,6,7,8,9,10]
ax.set_yticks(yticks)
ax.set_ylabel("Time per molecule (s)")
ax.grid(True)

ax = axes[1]
sns.boxplot(df.sort_values("time_docking_s", ascending=False), x="exhaustiveness", y="time_docking_s", hue="method", dodge=False, ax=ax)
ax.set_yscale("log")
# More yticks
yticks = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,2,4,5,6,7,8,9,10]
ax.set_yticks(yticks)
ax.set_ylabel("Docking Time")
ax.grid(True)

average_speedup = df.groupby(["method", "exhaustiveness"]).time_per_molecule_s.mean().reset_index()
average_speedup = average_speedup.pivot(index="exhaustiveness", columns="method", values="time_per_molecule_s")
average_speedup = (average_speedup["pyscreener"] / average_speedup["soft_docking"]).mean()
fig.suptitle(f"Speed Up: x{average_speedup:.1f}")
fig.tight_layout()

In [ ]:
# plot the scatterplot between pyscreener and autodock_gpu (API)

fig, axes = plt.subplots(1,2, figsize=(3*2,3))


for ax_lims, ax in zip([(None,), (-10, -6.5)],axes):
    sub_df = df.groupby(["target", "method", "commit_hash"]).scores.mean()
    sub_df = sub_df.reset_index()

    scores_pyscreener = sub_df[sub_df.method=="pyscreener"].scores.tolist()
    scores_autodock = sub_df[
      sub_df.method=="soft_docking"
    ].scores.tolist()

    sns.regplot(x=scores_pyscreener, y=scores_autodock, ax=ax)
    ax.set_xlabel("Scores (pyscreener)")
    ax.set_ylabel("Scores (autodock_gpu)")
    ax.set_xlim(*ax_lims)
    ax.set_ylim(*ax_lims)

fig.tight_layout()
